In [60]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

looked for top five words shared by Attempts to collect and Incorrect Info cols

In [61]:
complaints = pd.read_csv('complaints.csv')
incorrect_info = pd.read_csv('IncorrectInfoWords.csv')
attempts_collect = pd.read_csv('AttemptsToCollect.csv')

In [62]:
incorrect_attempts_merge = pd.concat([incorrect_info ,attempts_collect],axis=1)

In [63]:
incorrect_attempts_merge.head(6)

,word,count,word,count
0,credit,576152,credit,138483
1,account,348281,debt,128909
2,report,342119,account,105018
3,information,292415,information,69883
4,reporting,233173,report,65370
5,consumer,154393,collection,57077


lowercased words, removed Xs, factorized cols, renamed Consumer col

In [64]:
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].str.lower()

In [65]:
complaints_no_x = complaints['Consumer complaint narrative']

df_no_x = complaints_no_x.str.replace('xx', '').to_frame()

In [66]:
df = pd.concat([df_no_x, complaints['Issue']], axis=1)

In [67]:
df['issue_id'] = df['Issue'].factorize()[0]

In [68]:
df = df.rename(columns = {'Consumer complaint narrative': 'Consumer_complaint_narrative'})

pulled out Attempts to collect and Incorrect Info cols, replaced top five words shared with empty str 

In [69]:
df_incor_att = df.loc[(df['Issue'] == 'Attempts to collect debt not owed') | (df['Issue'] == 'Incorrect information on your report')]

In [70]:
df_incor_att['Consumer_complaint_narrative'] = df_incor_att['Consumer_complaint_narrative'].str.replace(r'\b(?:credit|account|report|reporting|information)\b', '', regex=True)

C:\Users\tjsim\AppData\Local\Temp\ipykernel_17768\3964784198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incor_att['Consumer_complaint_narrative'] = df_incor_att['Consumer_complaint_narrative'].str.replace(r'\b(?:credit|account|report|reporting|information)\b', '', regex=True)


ran linearSVC model with edited Attempts to Collect and Incorrect Info cols  

In [71]:
# only attempts and incorrect from Issue col 

X_train, X_test, y_train, y_test = train_test_split(df_incor_att['Consumer_complaint_narrative'], 
                                                    df_incor_att['Issue'], 
                                                    test_size=0.2, 
                                                    random_state=42) 


vectorizer = TfidfVectorizer(ngram_range=(1,3))


X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

clf = LinearSVC()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.88      0.83      0.85     14622
Incorrect information on your report       0.95      0.96      0.95     45872

                            accuracy                           0.93     60494
                           macro avg       0.91      0.90      0.90     60494
                        weighted avg       0.93      0.93      0.93     60494



removed Attempts to Collect and Incorrect Info cols from original df, concated edited version of those same cols

In [72]:
df_no_incor_att = df.loc[(df['Issue'] != 'Attempts to collect debt not owed') & (df['Issue'] != 'Incorrect information on your report')]

In [73]:
the_merge = pd.concat([df_no_incor_att, df_incor_att], ignore_index = True)

second linearSVC model ran on full df with edited Attempts to Collect and Incorrect Info cols replacing original versions. 

In [74]:
# using the merge - edited attempts and incorrect classes from Issue col concated with original untouched df

X_train, X_test, y_train, y_test = train_test_split(the_merge['Consumer_complaint_narrative'], 
                                                    the_merge['Issue'], 
                                                    test_size=0.2, 
                                                    random_state=42) 


vectorizer = TfidfVectorizer(ngram_range=(1,3))


X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

clf = LinearSVC()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.85      0.80      0.83     14609
               Communication tactics       0.90      0.88      0.89      4279
                       Fraud or scam       0.96      0.93      0.95      2527
Incorrect information on your report       0.94      0.96      0.95     45837
          Struggling to pay mortgage       0.95      0.96      0.95      3435

                            accuracy                           0.92     70687
                           macro avg       0.92      0.91      0.91     70687
                        weighted avg       0.92      0.92      0.92     70687



original linearSVC model ran on untouched df

In [75]:
# OG model

X_train, X_test, y_train, y_test = train_test_split(df['Consumer_complaint_narrative'], 
                                                    df['Issue'], 
                                                    test_size=0.2, 
                                                    random_state=42) 
vectorizer = TfidfVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

clf = LinearSVC()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.78      0.71      0.74     14477
               Communication tactics       0.84      0.76      0.80      4160
                       Fraud or scam       0.92      0.89      0.91      2391
Incorrect information on your report       0.91      0.95      0.93     46122
          Struggling to pay mortgage       0.93      0.93      0.93      3537

                            accuracy                           0.89     70687
                           macro avg       0.88      0.85      0.86     70687
                        weighted avg       0.88      0.89      0.88     70687

